# Analysis of adjancency matrices

In [13]:
import numpy as np

KGTN_PATH = '/tf/KGTN2'

In [14]:
distances_hierarchy = np.load(f'{KGTN_PATH}/KnowledgeGraphMatrix/HierarchyGraph.npy')
distances_glove = np.load(f'{KGTN_PATH}/KnowledgeGraphMatrix/WikidataGraph.npy')
distances_wikidata = np.load(f'{KGTN_PATH}/KnowledgeGraphMatrix/WikidataGraph.npy')